# 在 ollama-python 中使用 Ollama API
> 编者按：推荐使用官方库[ollama](2.%E5%9C%A8%20Python%20%E4%B8%AD%E4%BD%BF%E7%94%A8%20Ollama%20API.md)
## 一、 环境准备
在开始使用 Python 与 Ollama API 交互之前，请确保您的开发环境满足以下条件：

* Python: 安装 Python 3.9 或更高版本。
* pip: 确保已安装 pip，Python 的包管理工具。
* ollama-python 库: 用于更方便地与 Ollama API 交互。安装命令如下：

In [ ]:
pip install ollama-python

## 二、 基本请求示例
以下是一个使用 Python 发送请求到 Ollama API 的基本示例。
### 对话

In [14]:
import requests
import json

# Ollama API 的基础 URL
api_base_url = "http://localhost:11434/api/generate"

# 准备请求数据
data = {
    "model": "llama3.1",  # 指定模型名称
    "prompt": "Why is the sky blue?"  # 提供的提示
}

try:
    # 发送 POST 请求
    response = requests.post(api_base_url, json=data)
    
    if response.status_code == 200:
        full_response = ""
        for line in response.text.splitlines():
            try:
                completion = json.loads(line)
                # 累积响应片段
                full_response += completion["response"]
            except json.JSONDecodeError as e:
                print("Failed to decode JSON on this line:", line, e)
        
        # 打印完整的响应
        print("Full Response:", full_response)
    else:
        print("Failed to get response. Status code:", response.status_code)

except requests.exceptions.RequestException as e:
    print("An error occurred:", e)


Full Response: The sky appears blue to us because of a phenomenon called Rayleigh scattering. Here's why:

1. **Light from the sun**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Scattering by tiny particles**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules are much smaller than the wavelength of light.
3. **Rayleigh scattering**: The small gas molecules scatter the shorter (blue) wavelengths of light more efficiently than the longer (red) wavelengths. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh who first described it in 1871.
4. **Blue light dominates**: As a result of this scattering, the blue light is distributed throughout the atmosphere and reaches our eyes from all directions. Our brains then interpret this scattered blue light as the color of the sky.



### 2.1 流式响应处理
如果 API 配置为流式响应，你可能需要处理多个响应对象。以下是一个处理流式响应的示例。

In [15]:
from requests.exceptions import StreamConsumedError

# 发送请求，并设置 stream 参数为 true 以接收流式响应
response = requests.post(api_base_url, json=data, stream=True)

try:
    # 尝试从响应中读取内容
    for line in response.iter_lines():
        if line:
            # 解析每一行的 JSON 数据
            completion_part = json.loads(line.decode())
            print("Partial Response:", completion_part)
except StreamConsumedError:
    print("The stream has already been consumed.")

Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.499757919Z', 'response': 'The', 'done': False}
Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.52316723Z', 'response': ' sky', 'done': False}
Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.545883446Z', 'response': ' appears', 'done': False}
Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.569970608Z', 'response': ' blue', 'done': False}
Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.592916838Z', 'response': ' to', 'done': False}
Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.61617839Z', 'response': ' us', 'done': False}
Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.639182112Z', 'response': ' because', 'done': False}
Partial Response: {'model': 'llama3.1', 'created_at': '2024-08-13T02:20:55.661647692Z', 'response': ' of', 'done': False}
Partial Respo

### 2.2 高级参数
Ollama API 支持多种高级参数，我们可以在请求中使用它们来定制模型的行为。

In [16]:
# 包含高级参数的请求数据
data_with_options = {
    "model": "llama3.1",
    "prompt": "Why is the grass green?",
    "options": {
        "temperature": 0.9  # 设置温度参数，影响生成文本的随机性
    },
    "stream": False  # 关闭流式响应
}

# 发送请求
response = requests.post(api_base_url, json=data_with_options)

# 处理响应
if response.ok:
    completion = response.json()
    print("Response:", completion["response"])


Response: The color of grass being green is due to a combination of biology and chemistry. Here's a brief explanation:

**Chlorophyll:** The main reason grass appears green is because of a pigment called chlorophyll, which is present in the cells of plant leaves (including grass blades). Chlorophyll helps plants absorb sunlight, particularly blue light and red light, to undergo photosynthesis – the process by which they convert carbon dioxide and water into glucose (energy) and oxygen. Chlorophyll absorbs blue light and reflects green light, making it visible to our eyes.

**Pigment composition:** Grass contains other pigments as well, including carotenoids and anthocyanins. These pigments contribute to the overall color of grass by:

* Carotenoids: These yellow-pigmented compounds are present in chloroplasts (the green part of plant cells) and help protect plants from excessive light energy.
* Anthocyanins: These purple-pigmented compounds can appear in certain types of plants, like s

### 2.3 异常处理
在使用 API 时，正确处理可能发生的错误是非常重要的。

In [17]:
try:
    response = requests.post(api_base_url, json=data)
    response.raise_for_status()  # 如果响应状态码不是 200，将抛出异常
except requests.exceptions.HTTPError as err:
    print("HTTP error occurred:", err)
except Exception as err:
    print("An error occurred:", err)

### 2.4 完整工作示例
最后，我们可以将所有这些组合在一起，创建一个 Python 脚本，该脚本可以发送请求到 Ollama API 并处理响应。

In [18]:
import requests
import json

def generate_completion(model, prompt, options=None, stream=False):
    url = "http://localhost:11434/api/generate"
    headers = {"Content-Type": "application/json"}
    data = {"model": model, "prompt": prompt, "stream": stream}
    if options:
        data["options"] = options

    try:
        response = requests.post(url, json=data, headers=headers, stream=stream)
        response.raise_for_status()
        
        if stream:
            for line in response.iter_lines():
                if line:
                    print(json.loads(line.decode()))
        else:
            return response.json()
    
    except requests.exceptions.RequestException as e:
        print(e)
        return None

# 使用示例
if __name__ == "__main__":
    model_name = "llama3.1"
    prompt_text = "Why is the sky blue?"
    completion = generate_completion(model_name, prompt_text)
    if completion:
        print(completion)


{'model': 'llama3.1', 'created_at': '2024-08-13T02:21:57.267788394Z', 'response': "The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. **Sunlight**: The sun emits all colors of light (white light), which is made up of a spectrum of colors including red, orange, yellow, green, blue, indigo, and violet.\n2. **Scattering**: When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the light in all directions.\n3. **Short wavelengths scattered more**: The smaller molecules are better at scattering shorter wavelengths of light, which means they scatter blue light more than any other color. This is because blue light has a shorter wavelength than red or orange light, making it more susceptible to being scattered.\n4. **Blue dominates**: As the scattered blu

这个脚本提供了一个基本的框架，你可以根据需要添加更多的功能和错误处理逻辑。希望这个示例能帮助你开始在 Python 中使用 Ollama API。

## 三、 使用 ollama-python 库
### 3.1 ollama-python 库简介
ollama-python 库[（官方文档）](https://pypi.org/project/ollama-python/)提供了与 Ollama API 交互的高级接口，支持以下核心功能：

* 模型管理端点: 创建、删除、拉取、推送和列出模型等。
* 生成端点: 包括生成和聊天生成端点。
* 嵌入端点: 生成给定文本的嵌入。

### 3.2 使用OllamaPython 库
OllamaPython 库提供了一个更简洁的接口来与 Ollama API 交互。
以下是一个使用 `ollama-python` 库的完整 Python 脚本示例：


In [37]:
# 引入所需的模块
from ollama_python.endpoints import GenerateAPI, ModelManagementAPI
import json

# 定义函数来初始化 API 客户端
def initialize_api_client(base_url, model_name):
    # 根据提供的基本信息初始化 GenerateAPI 和 ModelManagementAPI 客户端
    generate_api = GenerateAPI(base_url=base_url, model=model_name)
    model_management_api = ModelManagementAPI(base_url=base_url)
    return generate_api, model_management_api

# 主函数
def main():
    # 设置 API 的基础 URL 和使用的模型名称
    base_url = "http://localhost:11434/api"
    model_name = "llama3.1"
    
    # 初始化 API 客户端
    generate_api, model_management_api = initialize_api_client(base_url, model_name)
    
    # 使用 GenerateAPI 客户端生成文本补全
    try:
        prompt = "Why is the sky blue?"
        response = generate_api.generate(prompt=prompt)
        print(f"Generated Completion for '{prompt}': {response.response}")
    except Exception as e:
        print(f"An error occurred while generating completion: {e}")
    
    # 使用 ModelManagementAPI 客户端列出本地模型
    try:
        models = model_management_api.list_local_models()
        print("Local Models:")
        for model in models.models:
            print(model)
    except Exception as e:
        print(f"An error occurred while listing local models: {e}")

# 检查脚本是否作为主程序运行
if __name__ == "__main__":
    main()

Generated Completion for 'Why is the sky blue?': The sky appears blue to us because of a phenomenon called scattering, which occurs when sunlight interacts with the tiny molecules of gases in the atmosphere. Here's a simplified explanation:

1. **Sunlight**: The sun emits white light, which contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Scattering**: When sunlight enters the Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the light in all directions.
3. **Shorter wavelengths scattered more**: The shorter wavelengths of light, like blue and violet, are scattered more than the longer wavelengths, like red and orange. This is because the smaller molecules are more effective at scattering the shorter wavelengths.
4. **Blue light dominates**: As a result of this scattering, the blue light becomes the dominant color that reaches our eyes from the sky.

There 

此脚本展示了如何使用 `ollama-python` 库进行基本操作：

1. **初始化 API 客户端**：创建 `GenerateAPI` 和 `ModelManagementAPI` 的实例。
2. **生成文本补全**：使用 `GenerateAPI` 来生成给定提示的文本补全。
3. **列出本地模型**：使用 `ModelManagementAPI` 来列出所有本地可用的模型。




### 3.3 示例用法
1. 补全（生成）
- 无流式传输

In [41]:
from ollama_python.endpoints import GenerateAPI

api = GenerateAPI(base_url="http://localhost:11434/api", model="llama3.1")
result = api.generate(prompt="Hello World")
result.response

'Hello! How are you today? Is there something I can help you with or would you like to chat?'

- 流式传输

In [44]:
from ollama_python.endpoints import GenerateAPI

# 初始化 GenerateAPI 客户端
api = GenerateAPI(base_url="http://localhost:11434/api", model="llama3.1")
# 生成文本，使用流式传输
for res in api.generate(prompt="Hello World", stream=True):
    print(res.response)  # 打印流式响应

Hello
!


Is
 there
 something
 I
 can
 help
 you
 with
,
 or
 would
 you
 like
 to
 chat
?



2. 聊天补全
* 无流式传输

In [3]:
from ollama_python.endpoints import GenerateAPI
from pydantic import ValidationError

# 初始化 GenerateAPI 客户端
api = GenerateAPI(base_url="http://localhost:11434/api", model="llama3.1")

# 准备聊天消息
messages = [{'role': 'user', 'content': '为什么天空是蓝色的？'}]

try:
    # 尝试生成聊天补全，不使用流式传输
    result = api.generate_chat_completion(messages=messages)
    print("Generated Completion:", result)

except ValidationError as e:
    print(f"Validation error occurred: {e}")

except Exception as e:
    print(f"An error occurred: {e}")

Validation error occurred: 2 validation errors for ChatCompletion
context
  Field required [type=missing, input_value={'model': 'llama3.1', 'cr...al_duration': 640337000}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
message
  Input should be a valid list [type=list_type, input_value={'role': 'assistant', 'co...的光线的颜色。'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/list_type


* 流式传输



In [4]:
from ollama_python.endpoints import GenerateAPI

# 初始化 GenerateAPI 客户端
api = GenerateAPI(base_url="http://localhost:11434/api", model="llama3.1")
# 准备聊天消息
messages = [{'role': 'user', 'content': '为什么天空是蓝色的？'}]
# 生成聊天补全，使用流式传输
for res in api.generate_chat_completion(messages=messages, stream=True):
    print(res.message)  # 打印流式响应中的聊天消息

ValidationError: 1 validation error for StreamChatCompletion
message
  Input should be a valid list [type=list_type, input_value={'role': 'assistant', 'content': '蓝'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/list_type

3. 带图片的聊天请求

In [ ]:
from ollama_python.endpoints import GenerateAPI

# 初始化 GenerateAPI 客户端，使用支持多模态的模型
api = GenerateAPI(base_url="http://localhost:11434/api", model="llava")
# 准备包含图片的聊天消息
messages = [{'role': 'user', 'content': '这张图片里有什么？', 'image': '图片的Base64编码'}]
# 生成聊天补全，处理图片
result = api.generate_chat_completion(messages=messages)
print(result.message)  # 打印包含图片处理结果的聊天消息

4. 嵌入（Embeddings Endpoint）
* 生成嵌入
> 注意：此端点已被 `/api/embed` 取代

In [ ]:
from ollama_python.endpoints import EmbeddingAPI

# 初始化 EmbeddingAPI 客户端
api = EmbeddingAPI(base_url="http://localhost:11434/api", model="llama3.1")
# 生成文本嵌入，设置随机种子以复现结果
result = api.get_embedding(prompt="Hello World", options={"seed": 10})

5. 模型管理
* 创建模型
  
无流式传输

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 创建模型
result = api.create(name="test_model", model_file="模型文件路径")

流式传输

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 创建模型，使用流式传输
for res in api.create(name="test_model", model_file="模型文件路径", stream=True):
    print(res.status)  # 打印创建过程中的状态信息

* 列出本地模型

In [3]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 列出本地模型
result = api.list_local_models()
print(result.models)  # 打印本地模型列表

[ModelTag(name='llama3.1:latest', digest='91ab477bec9d27086a119e33c471ae7afbd786cc4fbd8f38d8af0a0b949d53aa', size=4661230977, modified_at='2024-08-09T17:49:08.348175042+08:00', details=ModelDetails(format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0')), ModelTag(name='llava:latest', digest='8dd30f6b0cb19f555f2c7a7ebda861449ea2cc76bf1f44e262931f45fc81d081', size=4733363377, modified_at='2024-08-08T14:20:09.31045753+08:00', details=ModelDetails(format='gguf', family='llama', families=['llama', 'clip'], parameter_size='7B', quantization_level='Q4_0')), ModelTag(name='glm4xiuxian:latest', digest='348c4463f081b362ef39ef448ff73a1674f9cf55c703de0883c64afbd8cf34c6', size=4661231198, modified_at='2024-08-07T16:16:37.744493536+08:00', details=ModelDetails(format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0'))]


* 显示模型信息

In [5]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 显示模型信息
result = api.show(name="llama3.1")
print(result.details)  # 打印模型详细信息

format='gguf' family='llama' families=['llama'] parameter_size='8.0B' quantization_level='Q4_0'


* 复制模型

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 复制模型
result = api.copy(source="原模型名称", destination="复制的目标模型名称")

* 删除模型

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 删除模型
api.delete(name="要删除的模型名称")

* 拉取模型 
  
无流式传输

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 拉取模型
result = api.pull(name="模型名称")
print(result.status)  # 打印拉取状态

流式传输

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 拉取模型，使用流式传输
for res in api.pull(name="模型名称", stream=True):
    print(res.status)  # 打印流式传输中的拉取状态

* 推送模型

无流式传输

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 推送模型
result = api.push(name="模型名称")
print(result.status)  # 打印推送状态

流式传输

In [ ]:
from ollama_python.endpoints import ModelManagementAPI

# 初始化 ModelManagementAPI 客户端
api = ModelManagementAPI(base_url="http://localhost:11434/api")
# 推送模型，使用流式传输
for res in api.push(name="模型名称", stream=True):
    print(res.status)  # 打印流式传输中的推送状态

> 参考文档
* [ollama-python 库](https://pypi.org/project/ollama-python/)
* [Ollama python](https://github.com/ollama/ollama-python)